In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.fft import fft

# Dados
valores = [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 59, 210, 339, 443, 470, 456, 432, 453, 428, 433, 472, 498, 492, 476, 433, 355, 271, 247, 226, 203, 370, 296, 175, 78, 8, 0, 0, 0, 0, 0, 0, 0]
tempo = np.linspace(0, 24, len(valores))  # 24 horas

plt.figure(figsize=(10,6))
plt.scatter(tempo, valores, label='Dados', color='blue')
plt.title('Dados de luminosidade ao longo do tempo')
plt.xlabel('Tempo (horas)')
plt.ylabel('Luminosidade (unidades arbitrárias)')
plt.grid(True)
plt.legend()
plt.show()

# Função trapezoidal
def funcao_trapezoidal(x, a, b, c, d, h):
    return np.piecewise(x,
        [x < a, (a <= x) & (x < b), (b <= x) & (x < c), (c <= x) & (x < d), d <= x],
        [lambda x: 0,
         lambda x: h * (x - a) / (b - a),
         lambda x: h,
         lambda x: h * (1 - (x - c) / (d - c)),
         lambda x: 0])

# Ajuste da função trapezoidal aos dados
popt, _ = curve_fit(funcao_trapezoidal, tempo, valores,
                    p0=[5, 8, 16, 19, max(valores)],  # estimativas iniciais
                    bounds=([0, 0, 0, 0, 0], [24, 24, 24, 24, max(valores)]))

# Criação de pontos para a função ajustada
tempo_fino = np.linspace(0, 24, 1000)
valores_ajustados = funcao_trapezoidal(tempo_fino, *popt)

# 1. Plotar os pontos originais e a função ajustada
plt.figure(figsize=(12, 8))
plt.plot(tempo, valores, 'bo', label='Dados originais')
plt.plot(tempo_fino, valores_ajustados, 'r-', label='Função trapezoidal ajustada')
plt.title('Leituras do Sensor LDR e Função Trapezoidal Ajustada')
plt.xlabel('Tempo (horas)')
plt.ylabel('Valor do Sensor')
plt.legend()
plt.grid(True)
plt.show()

# 2. Transformada de Fourier dos pontos originais
fft_valores = fft(valores)
frequencias = np.fft.fftfreq(len(tempo), tempo[1] - tempo[0])

plt.figure(figsize=(12, 8))
plt.plot(frequencias[:len(frequencias)//2], np.abs(fft_valores)[:len(frequencias)//2])
plt.title('Transformada de Fourier dos Dados Originais')
plt.xlabel('Frequência')
plt.ylabel('Magnitude')
plt.grid(True)
plt.show()

# 3. Transformada de Fourier da função ajustada
fft_ajustados = fft(valores_ajustados)
frequencias_ajustadas = np.fft.fftfreq(len(tempo_fino), tempo_fino[1] - tempo_fino[0])

plt.figure(figsize=(12, 8))
plt.plot(frequencias_ajustadas[:len(frequencias_ajustadas)//2], np.abs(fft_ajustados)[:len(frequencias_ajustadas)//2])
plt.title('Transformada de Fourier da Função Trapezoidal Ajustada')
plt.xlabel('Frequência')
plt.ylabel('Magnitude')
plt.grid(True)
plt.show()

# Definindo um limiar para ativar o motor durante o dia
limiar = 100 
motor_ativo = valores_ajustados > limiar

plt.figure(figsize=(12, 8))
plt.plot(tempo_fino, valores_ajustados, 'b-', label='Função ajustada')
plt.fill_between(tempo_fino, 0, max(valores_ajustados), where=motor_ativo, alpha=0.3, color='g', label='Motor ativo')
plt.axhline(y=limiar, color='r', linestyle='--', label='Limiar')
plt.title('Ativação do Motor Baseada na Função Trapezoidal')
plt.xlabel('Tempo (horas)')
plt.ylabel('Valor do Sensor')
plt.legend()
plt.grid(True)
plt.show()

# Imprimir os parâmetros ajustados
print(f"Parâmetros ajustados: a={popt[0]:.2f}, b={popt[1]:.2f}, c={popt[2]:.2f}, d={popt[3]:.2f}, h={popt[4]:.2f}")
